
# Functional Programming in Python

Functional programming is a programming paradigm that focuses on using functions to perform computation and avoiding changing state and mutable data. It is a declarative programming style, where the programmer specifies what the program should do, instead of how it should do it. This leads to more concise, easy to understand, and predictable code.

In Python, functional programming can be achieved through the use of higher-order functions such as map, filter, and reduce. These functions can be used to perform operations on lists or other iterable objects in a functional manner. Another way to implement functional programming in Python is through the use of lambda functions. These are small anonymous functions that can be defined inline and passed as arguments to other functions, similar to closures or function pointers in other languages. They are useful for defining simple, one-time-use functions that would be impractical to define using the def keyword.

As is generally the case with Python, it can be used to write functional code, but there is no way to enforce the standard. Other languages, like Haskell, were designed to be functional programming languages and have the tenants of functional programming built in.


Core tenants:
* You always work with immutable data structures. One of the benefits of this is that it enables you to more easily parallelize code because your data structures are locked, so you don't have to worry about simultaneous reads from different threads
* Decrease errors because you do not have to retype field names

Pros:
* Runs faster
* Can decrease bugs

Cons:
* Steep learning curve
* Verbosity: functional programming can lead to more verbose code, as functions are often used in place of simpler constructs like loops and conditional statements.
* Harder to debug: since functional programming emphasizes immutability and the avoidance of side-effects, it can be harder to debug issues that arise in functional code.


## Contents

* [Immutable Data Structures](#immutable_data_structures)
* [Functional Primatives](#Functional-Primatives)
* [filter()](#filter())
* [map()](#map())
* [reduce()](#reduce())
* [Parallel Processing with Functional Programming](#Parallel-Processing-with-Functional-Programming)
* [Multiprocessing](#Multiprocessing)
* [Concurrent.futures](#concurrent.futures)
* [Multiprocessing vs Concurrent.Futures and the GIL](#Multiprocessing-vs-Concurrent.Futures-and-the-GIL)
* [The End of the GIL?](#The-End-of-the-GIL?)
* [Resources](#Resources)


Data cleaning I did on underlying [datasource](https://www.kaggle.com/datasets/matheusdalbuquerque/philosophers-dataset?resource=download) saved for reference 

In [1]:
import pandas as pd
import ast
import re

df = pd.read_csv("Philosophers-dataset.csv")

new_df = df[["Born", "Birth place", "Died", "Subjects Of Study", "Philosophers"]]
new_df = new_df.dropna()
new_df = new_df[new_df["Born"].str.contains("BCE") == False] # Don't want to deal with BCE dates; sorry Socrates


def get_country(places):
    if not places:
        return None
    else:
        return places[-1]
    
new_df["Birth Country"] = new_df['Birth place'].apply(lambda x: ast.literal_eval(x)).apply(get_country)
# This is ugly, but I got a bit lazy; will endeavor to beautify later
new_df["Born"] = new_df["Born"].apply(lambda x: x.split(", ")[-1]).apply(lambda x: x.split(". ")[-1]).apply(lambda x: x.split(" ")[-1]).apply(lambda x: x.replace("?", "")).apply(lambda x: int(x))
new_df["Died"] = new_df["Died"].apply(lambda x: x.split(", ")[-1]).apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x)).apply(lambda x: x.split(". ")[-1]).apply(lambda x: x.split(" ")[-1] if x.split(" ")[-1] != "" else x.split(" ")[0]).apply(lambda x: x.replace(")", "")).apply(lambda x: x.replace("?", "")).apply(lambda x: int(x))
del new_df["Birth place"]
new_df.columns = ["_".join(i.lower().split(" ")) for i in new_df.columns]
new_df = new_df.dropna()
new_df = new_df.rename(columns={"philosophers": "name"})

# We'll need this for the reduce section later 
all_subjects = list(set([a for b in new_df["subjects_of_study"].apply(lambda x: x.split(", ")).tolist() for a in b]))
all_countries = list(set([x for x in new_df["birth_country"].tolist()]))

new_df



,born,died,subjects_of_study,name,birth_country
1,1856,1922,"Agriculture, Zionism",Aaron David Gordon,Ukraine
3,1907,1972,"Judaism, Philosophy Of Religion",Abraham Joshua Heschel,Poland
18,1316,1390,Gravity,Albert of Saxony,Germany
23,1463,1512,William Of Ockham,Alessandro Achillini,Italy
26,1714,1762,"Aesthetics, Gottfried Wilhelm Leibniz, Feeling...",Alexander Gottlieb Baumgarten,Germany
...,...,...,...,...,...
655,1020,1077,"Ren, Qi",Zhang Zai,China
656,1017,1073,"Confucianism, Dark Learning",Zhou Dunyi,China
657,1130,1200,"Qi, Li",Zhu Xi,China
661,878,950,"Political Philosophy, Greek Philosophy",al-Farabi,Turkistan


<a id='immutable_data_structures'></a>
## Immutable Data Structures

In [2]:
# Namedtuples
import collections
from pprint import pprint


Philosopher = collections.namedtuple("Philosopher", [
    "born",
    "died",
    "subjects_of_study",
    "name",
    "birth_country"
])


# This could also been defined:
# Philosophers = collections.namedtuple("Philosopher", [
#     *new_df.columns
# ])
# 
# This is in fact the preferred way to define it to ensure you are not getting the order wrong in the step below


In [3]:
# mixed mutable and immutable types

philosophers = [Philosopher(*row) for row in new_df.itertuples(index=False, name=None)]
philosophers

[Philosopher(born=1856, died=1922, subjects_of_study='Agriculture, Zionism', name='Aaron David Gordon', birth_country='Ukraine'),
 Philosopher(born=1907, died=1972, subjects_of_study='Judaism, Philosophy Of Religion', name='Abraham Joshua Heschel', birth_country='Poland'),
 Philosopher(born=1316, died=1390, subjects_of_study='Gravity', name='Albert of Saxony', birth_country='Germany'),
 Philosopher(born=1463, died=1512, subjects_of_study='William Of Ockham', name='Alessandro Achillini', birth_country='Italy'),
 Philosopher(born=1714, died=1762, subjects_of_study='Aesthetics, Gottfried Wilhelm Leibniz, Feeling, Christian, Baron Von Wolff', name='Alexander Gottlieb Baumgarten', birth_country='Germany'),
 Philosopher(born=1853, died=1920, subjects_of_study='Objectivism, Intentionality', name='Alexius Meinong', birth_country='Ukraine'),
 Philosopher(born=1857, died=1940, subjects_of_study='Philosophy Of Religion, Modernism', name='Alfred Firmin Loisy', birth_country='France'),
 Philosopher

In [4]:
philosophers[0]

Philosopher(born=1856, died=1922, subjects_of_study='Agriculture, Zionism', name='Aaron David Gordon', birth_country='Ukraine')

This definitely worked, but it is not what we need for our purposes because lists are mutable in Python

In [5]:
philosophers[0] = Philosopher(born=0, died=50, subjects_of_study="Nothing", name="Jim", birth_country="Unknown")
philosophers[0]

Philosopher(born=0, died=50, subjects_of_study='Nothing', name='Jim', birth_country='Unknown')

Instead, we should use tuples:

In [6]:
philosophers = tuple(Philosopher(*row) for row in new_df.itertuples(index=False, name=None))
philosophers[0] = Philosopher(born=0, died=50, subjects_of_study="Nothing", name="Jim", birth_country="Unknown")

TypeError: 'tuple' object does not support item assignment

# Functional Primatives

filter, map, and reduce

## filter()



In [7]:
help(filter)

Help on class filter in module builtins:

class filter(object)
 |  filter(function or None, iterable) --> filter object
 |  
 |  Return an iterator yielding those items of iterable for which function(item)
 |  is true. If function is None, return the items that are true.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



What the hell does that mean?

In [8]:
filter(lambda x: x.subjects_of_study.contains("Existentialism"), philosophers)

In [9]:
fs = filter(lambda x: "Existentialism" in x.subjects_of_study, philosophers)

In [10]:
while True:
    try:
        print(next(fs))
    except:
        break

Philosopher(born=1886, died=1929, subjects_of_study='Judaism, Existentialism', name='Franz Rosenzweig', birth_country='Germany')
Philosopher(born=1905, died=1980, subjects_of_study='Marxism, Existentialism, Phenomenology, Gustave Flaubert, Littérature Engagée', name='Jean-Paul Sartre', birth_country='France')
Philosopher(born=1910, died=2009, subjects_of_study='Evolution, Philosophy Of Biology, Existentialism, Martin Heidegger, Karl Jaspers', name='Marjorie Grene', birth_country='Wisconsin')
Philosopher(born=1884, died=1976, subjects_of_study='Theological Existentialism', name='Rudolf Bultmann', birth_country='Germany')
Philosopher(born=1889, died=1960, subjects_of_study='Ethics, Buddhism, Existentialism', name='Watsuji Tetsuro', birth_country='Japan')


In [11]:
next(fs)

StopIteration: 

In [12]:
fs = filter(lambda x: "Existentialism" in x.subjects_of_study, philosophers)
tuple(fs)

(Philosopher(born=1886, died=1929, subjects_of_study='Judaism, Existentialism', name='Franz Rosenzweig', birth_country='Germany'),
 Philosopher(born=1905, died=1980, subjects_of_study='Marxism, Existentialism, Phenomenology, Gustave Flaubert, Littérature Engagée', name='Jean-Paul Sartre', birth_country='France'),
 Philosopher(born=1910, died=2009, subjects_of_study='Evolution, Philosophy Of Biology, Existentialism, Martin Heidegger, Karl Jaspers', name='Marjorie Grene', birth_country='Wisconsin'),
 Philosopher(born=1884, died=1976, subjects_of_study='Theological Existentialism', name='Rudolf Bultmann', birth_country='Germany'),
 Philosopher(born=1889, died=1960, subjects_of_study='Ethics, Buddhism, Existentialism', name='Watsuji Tetsuro', birth_country='Japan'))

Why do it like this? Why not do a for loop?

filter() allows for chaining of function calls which is more declarative

In [13]:
def existential_filter(x):
    return "Existentialism" in x.subjects_of_study

print(tuple(filter(existential_filter, philosophers)))

# This is not necessarily how Python was designed to be used, but is a useful paradigm to expand how you think about
# writing your code

(Philosopher(born=1886, died=1929, subjects_of_study='Judaism, Existentialism', name='Franz Rosenzweig', birth_country='Germany'), Philosopher(born=1905, died=1980, subjects_of_study='Marxism, Existentialism, Phenomenology, Gustave Flaubert, Littérature Engagée', name='Jean-Paul Sartre', birth_country='France'), Philosopher(born=1910, died=2009, subjects_of_study='Evolution, Philosophy Of Biology, Existentialism, Martin Heidegger, Karl Jaspers', name='Marjorie Grene', birth_country='Wisconsin'), Philosopher(born=1884, died=1976, subjects_of_study='Theological Existentialism', name='Rudolf Bultmann', birth_country='Germany'), Philosopher(born=1889, died=1960, subjects_of_study='Ethics, Buddhism, Existentialism', name='Watsuji Tetsuro', birth_country='Japan'))


Pythonic replacement for filtering

In [14]:
tuple(x for x in philosophers if "Existentialism" in x.subjects_of_study)

(Philosopher(born=1886, died=1929, subjects_of_study='Judaism, Existentialism', name='Franz Rosenzweig', birth_country='Germany'),
 Philosopher(born=1905, died=1980, subjects_of_study='Marxism, Existentialism, Phenomenology, Gustave Flaubert, Littérature Engagée', name='Jean-Paul Sartre', birth_country='France'),
 Philosopher(born=1910, died=2009, subjects_of_study='Evolution, Philosophy Of Biology, Existentialism, Martin Heidegger, Karl Jaspers', name='Marjorie Grene', birth_country='Wisconsin'),
 Philosopher(born=1884, died=1976, subjects_of_study='Theological Existentialism', name='Rudolf Bultmann', birth_country='Germany'),
 Philosopher(born=1889, died=1960, subjects_of_study='Ethics, Buddhism, Existentialism', name='Watsuji Tetsuro', birth_country='Japan'))

## map()



In [15]:
help(map)

Help on class map in module builtins:

class map(object)
 |  map(func, *iterables) --> map object
 |  
 |  Make an iterator that computes the function using arguments from
 |  each of the iterables.  Stops when the shortest iterable is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [16]:
names_and_ages = tuple(map(lambda x: {"name": x.name, "age": x.died - x.born}, philosophers))
names_and_ages

({'name': 'Aaron David Gordon', 'age': 66},
 {'name': 'Abraham Joshua Heschel', 'age': 65},
 {'name': 'Albert of Saxony', 'age': 74},
 {'name': 'Alessandro Achillini', 'age': 49},
 {'name': 'Alexander Gottlieb Baumgarten', 'age': 48},
 {'name': 'Alexius Meinong', 'age': 67},
 {'name': 'Alfred Firmin Loisy', 'age': 83},
 {'name': 'Alfred Korzybski', 'age': 71},
 {'name': 'Alfred North Whitehead', 'age': 86},
 {'name': 'Allan Bloom', 'age': 62},
 {'name': 'Anicius Manlius Severinus Boethius', 'age': 49},
 {'name': 'Antonio Labriola', 'age': 61},
 {'name': 'Antonio Rosmini-Serbati', 'age': 58},
 {'name': 'Antony Flew', 'age': 87},
 {'name': 'Armando Carlini', 'age': 81},
 {'name': 'Arthur O. Lovejoy', 'age': 89},
 {'name': 'Arthur Schopenhauer', 'age': 72},
 {'name': 'Arthur Stanley Eddington', 'age': 62},
 {'name': 'Ashvaghosha', 'age': 70},
 {'name': 'Auguste Comte', 'age': 59},
 {'name': 'Averroes', 'age': 72},
 {'name': 'Avicenna', 'age': 57},
 {'name': 'Bartolome de Medina', 'age': 5

In [17]:
# Pythonic version of above code

[{"name": x.name, "age": x.died - x.born} for x in philosophers]

[{'name': 'Aaron David Gordon', 'age': 66},
 {'name': 'Abraham Joshua Heschel', 'age': 65},
 {'name': 'Albert of Saxony', 'age': 74},
 {'name': 'Alessandro Achillini', 'age': 49},
 {'name': 'Alexander Gottlieb Baumgarten', 'age': 48},
 {'name': 'Alexius Meinong', 'age': 67},
 {'name': 'Alfred Firmin Loisy', 'age': 83},
 {'name': 'Alfred Korzybski', 'age': 71},
 {'name': 'Alfred North Whitehead', 'age': 86},
 {'name': 'Allan Bloom', 'age': 62},
 {'name': 'Anicius Manlius Severinus Boethius', 'age': 49},
 {'name': 'Antonio Labriola', 'age': 61},
 {'name': 'Antonio Rosmini-Serbati', 'age': 58},
 {'name': 'Antony Flew', 'age': 87},
 {'name': 'Armando Carlini', 'age': 81},
 {'name': 'Arthur O. Lovejoy', 'age': 89},
 {'name': 'Arthur Schopenhauer', 'age': 72},
 {'name': 'Arthur Stanley Eddington', 'age': 62},
 {'name': 'Ashvaghosha', 'age': 70},
 {'name': 'Auguste Comte', 'age': 59},
 {'name': 'Averroes', 'age': 72},
 {'name': 'Avicenna', 'age': 57},
 {'name': 'Bartolome de Medina', 'age': 5

In [18]:
# Pythonic version of above code in an immutable data structure (but dict still mutable, would need namedtuple
# for true immutability)

tuple({"name": x.name, "age": x.died - x.born} for x in philosophers)

({'name': 'Aaron David Gordon', 'age': 66},
 {'name': 'Abraham Joshua Heschel', 'age': 65},
 {'name': 'Albert of Saxony', 'age': 74},
 {'name': 'Alessandro Achillini', 'age': 49},
 {'name': 'Alexander Gottlieb Baumgarten', 'age': 48},
 {'name': 'Alexius Meinong', 'age': 67},
 {'name': 'Alfred Firmin Loisy', 'age': 83},
 {'name': 'Alfred Korzybski', 'age': 71},
 {'name': 'Alfred North Whitehead', 'age': 86},
 {'name': 'Allan Bloom', 'age': 62},
 {'name': 'Anicius Manlius Severinus Boethius', 'age': 49},
 {'name': 'Antonio Labriola', 'age': 61},
 {'name': 'Antonio Rosmini-Serbati', 'age': 58},
 {'name': 'Antony Flew', 'age': 87},
 {'name': 'Armando Carlini', 'age': 81},
 {'name': 'Arthur O. Lovejoy', 'age': 89},
 {'name': 'Arthur Schopenhauer', 'age': 72},
 {'name': 'Arthur Stanley Eddington', 'age': 62},
 {'name': 'Ashvaghosha', 'age': 70},
 {'name': 'Auguste Comte', 'age': 59},
 {'name': 'Averroes', 'age': 72},
 {'name': 'Avicenna', 'age': 57},
 {'name': 'Bartolome de Medina', 'age': 5

## reduce()

reduce has to be imported in Python3

In [19]:
from functools import reduce

help(reduce)

Help on built-in function reduce in module _functools:

reduce(...)
    reduce(function, iterable[, initial]) -> value
    
    Apply a function of two arguments cumulatively to the items of a sequence
    or iterable, from left to right, so as to reduce the iterable to a single
    value.  For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
    ((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
    of the iterable in the calculation, and serves as a default when the
    iterable is empty.



In [20]:
total_age = reduce(
    lambda acc, val: acc + val["age"],
    names_and_ages,
    0
)

In [21]:
total_age

19529

In [22]:
sum(x["age"] for x in names_and_ages)

19529

Why reduce? Because it goes deep and can be incredibly powerful

In [23]:
def country_reducer(acc, philosopher):
    acc[philosopher.birth_country].append(philosopher)
    return acc

all_countries_map = {k: [] for k in all_countries}

philosopher_by_country = reduce(
    country_reducer,
    philosophers,
    all_countries_map
)

for country, person in philosopher_by_country.items():
    print(f"{country}: \n\n{person}\n")

Portugal: 

[Philosopher(born=1585, died=1640, subjects_of_study='Rationalism, Religion', name='Uriel Acosta', birth_country='Portugal')]

Czechoslovakia: 

[Philosopher(born=1859, died=1938, subjects_of_study='Consciousness, Eidetic Reduction, Phenomenological Reduction', name='Edmund Husserl', birth_country='Czechoslovakia'), Philosopher(born=1901, died=1985, subjects_of_study='Logic, Scientific Method', name='Ernest Nagel', birth_country='Czechoslovakia')]

Turkey: 

[Philosopher(born=1403, died=1472, subjects_of_study='Greek Literature, Platonism', name='Bessarion', birth_country='Turkey'), Philosopher(born=1405, died=1473, subjects_of_study='Aristotelianism, Neoplatonism', name='Gennadios II Scholarios', birth_country='Turkey'), Philosopher(born=1355, died=1452, subjects_of_study='Aristotelianism, Neoplatonism', name='George Gemistus Plethon', birth_country='Turkey')]

Belgium: 

[Philosopher(born=1547, died=1606, subjects_of_study='Stoicism', name='Justus Lipsius', birth_country=

We can still make this cooler

In [24]:
philosopher_by_country = reduce(
    country_reducer,
    philosophers,
    collections.defaultdict(list)
)

for country, person in philosopher_by_country.items():
    print(f"{country}: \n\n{person}\n")

Ukraine: 

[Philosopher(born=1856, died=1922, subjects_of_study='Agriculture, Zionism', name='Aaron David Gordon', birth_country='Ukraine'), Philosopher(born=1853, died=1920, subjects_of_study='Objectivism, Intentionality', name='Alexius Meinong', birth_country='Ukraine'), Philosopher(born=1900, died=1975, subjects_of_study='Evolution, Speciation, Drosophila, Synthetic Theory Of Evolution', name='Theodosius Dobzhansky', birth_country='Ukraine')]

Poland: 

[Philosopher(born=1907, died=1972, subjects_of_study='Judaism, Philosophy Of Religion', name='Abraham Joshua Heschel', birth_country='Poland'), Philosopher(born=1879, died=1950, subjects_of_study='Semantics, General Semantics', name='Alfred Korzybski', birth_country='Poland'), Philosopher(born=1788, died=1860, subjects_of_study='Philosophy Of Art, Panpsychism', name='Arthur Schopenhauer', birth_country='Poland'), Philosopher(born=1891, died=1942, subjects_of_study='Phenomenology, Thomism', name='Edith Stein', birth_country='Poland'),

More Pythonic way? Maybe...

In [25]:
import itertools

philosopher_by_country = {
    item[0]: list(item[1])
    for item in itertools.groupby(philosophers, lambda x: x.birth_country)
}

for country, person in philosopher_by_country.items():
    print(f"{country}: \n\n{person}\n")

Ukraine: 

[Philosopher(born=1900, died=1975, subjects_of_study='Evolution, Speciation, Drosophila, Synthetic Theory Of Evolution', name='Theodosius Dobzhansky', birth_country='Ukraine')]

Poland: 

[Philosopher(born=1823, died=1891, subjects_of_study='Algebraic Equation, Algebraic Number, Elliptic Function', name='Leopold Kronecker', birth_country='Poland')]

Germany: 

[Philosopher(born=1833, died=1911, subjects_of_study='Social Science, Humanities, Methodology', name='Wilhelm Dilthey', birth_country='Germany')]

Italy: 

[Philosopher(born=1433, died=1499, subjects_of_study='Plato, Plotinus, Soul', name='Marsilio Ficino', birth_country='Italy')]

France: 

[Philosopher(born=1150, died=1231, subjects_of_study='Theology, Natural Law, Free Will, Liturgy', name='William of Auxerre', birth_country='France'), Philosopher(born=1070, died=1121, subjects_of_study='Universal', name='William of Champeaux', birth_country='France')]

England: 

[Philosopher(born=1285, died=1349, subjects_of_study

In [26]:
def reducer(acc, philosopher):
    subjects = philosopher.subjects_of_study.split(", ")
    for subject in subjects:
        acc[subject].append(philosopher)
    return acc

all_subject_map = {k: [] for k in all_subjects}

philosopher_by_field = reduce(
    reducer,
    philosophers,
    all_subject_map
)

for subject, person in philosopher_by_field.items():
    print(f"{subject}: \n\n{person}\n")

Emotion: 

[Philosopher(born=1822, died=1893, subjects_of_study='Thought, Emotion', name='Jacob Moleschott', birth_country='Netherlands'), Philosopher(born=226, died=249, subjects_of_study='Emotion, “Yijing”, “Tao-Te Ching”, Dark Learning', name='Wang Bi', birth_country='China')]

Marxism: 

[Philosopher(born=1905, died=1980, subjects_of_study='Marxism, Existentialism, Phenomenology, Gustave Flaubert, Littérature Engagée', name='Jean-Paul Sartre', birth_country='France'), Philosopher(born=1918, died=1990, subjects_of_study='Marxism, Structuralism', name='Louis Althusser', birth_country='Algeria'), Philosopher(born=1908, died=1961, subjects_of_study='Marxism', name='Maurice Merleau-Ponty', birth_country='France'), Philosopher(born=1902, died=1989, subjects_of_study='Marxism', name='Sidney Hook', birth_country='New York')]

Functionalism: 

[Philosopher(born=1842, died=1921, subjects_of_study='Biological Psychology, Functionalism, Experimental Psychology', name='George Trumbull Ladd', bi

## Parallel Processing with Functional Programming

### Multiprocessing

In [27]:
# Simple example; if this involved I/O wait or intense computations
def transform(x):
    return {"name": x.name, "age": x.died - x.born}

result = tuple(map(
    transform,
    philosophers
))

result

({'name': 'Aaron David Gordon', 'age': 66},
 {'name': 'Abraham Joshua Heschel', 'age': 65},
 {'name': 'Albert of Saxony', 'age': 74},
 {'name': 'Alessandro Achillini', 'age': 49},
 {'name': 'Alexander Gottlieb Baumgarten', 'age': 48},
 {'name': 'Alexius Meinong', 'age': 67},
 {'name': 'Alfred Firmin Loisy', 'age': 83},
 {'name': 'Alfred Korzybski', 'age': 71},
 {'name': 'Alfred North Whitehead', 'age': 86},
 {'name': 'Allan Bloom', 'age': 62},
 {'name': 'Anicius Manlius Severinus Boethius', 'age': 49},
 {'name': 'Antonio Labriola', 'age': 61},
 {'name': 'Antonio Rosmini-Serbati', 'age': 58},
 {'name': 'Antony Flew', 'age': 87},
 {'name': 'Armando Carlini', 'age': 81},
 {'name': 'Arthur O. Lovejoy', 'age': 89},
 {'name': 'Arthur Schopenhauer', 'age': 72},
 {'name': 'Arthur Stanley Eddington', 'age': 62},
 {'name': 'Ashvaghosha', 'age': 70},
 {'name': 'Auguste Comte', 'age': 59},
 {'name': 'Averroes', 'age': 72},
 {'name': 'Avicenna', 'age': 57},
 {'name': 'Bartolome de Medina', 'age': 5

In [28]:
%%time
import time 


def transform(x):
    print(f"Processing philosopher: {x.name}")
    time.sleep(.01)
    return {"name": x.name, "age": x.died - x.born}

result = tuple(map(
    transform,
    philosophers
))

result

Processing philosopher: Aaron David Gordon
Processing philosopher: Abraham Joshua Heschel
Processing philosopher: Albert of Saxony
Processing philosopher: Alessandro Achillini
Processing philosopher: Alexander Gottlieb Baumgarten
Processing philosopher: Alexius Meinong
Processing philosopher: Alfred Firmin Loisy
Processing philosopher: Alfred Korzybski
Processing philosopher: Alfred North Whitehead
Processing philosopher: Allan Bloom
Processing philosopher: Anicius Manlius Severinus Boethius
Processing philosopher: Antonio Labriola
Processing philosopher: Antonio Rosmini-Serbati
Processing philosopher: Antony Flew
Processing philosopher: Armando Carlini
Processing philosopher: Arthur O. Lovejoy
Processing philosopher: Arthur Schopenhauer
Processing philosopher: Arthur Stanley Eddington
Processing philosopher: Ashvaghosha
Processing philosopher: Auguste Comte
Processing philosopher: Averroes
Processing philosopher: Avicenna
Processing philosopher: Bartolome de Medina
Processing philosop

Processing philosopher: Oswald Kulpe
Processing philosopher: Oswald Spengler
Processing philosopher: Otto Friedrich von Gierke
Processing philosopher: Otto Neurath
Processing philosopher: Paul Shorey
Processing philosopher: Paul Tillich
Processing philosopher: Paul of Venice
Processing philosopher: Pierre Bayle
Processing philosopher: Pierre Gassendi
Processing philosopher: Pierre Laromiguiere
Processing philosopher: Pierre Leroux
Processing philosopher: Pierre Nicole
Processing philosopher: Pierre Teilhard de Chardin
Processing philosopher: Pierre-Felix Guattari
Processing philosopher: Pietro Pomponazzi
Processing philosopher: Raghunatha Shiromani
Processing philosopher: Ramana Maharshi
Processing philosopher: Ramanuja
Processing philosopher: Ramon Llull
Processing philosopher: Reinhold Niebuhr
Processing philosopher: Rene Descartes
Processing philosopher: Richard Hooker
Processing philosopher: Richard Price
Processing philosopher: Robert Boyle
Processing philosopher: Robert Grossetes

({'name': 'Aaron David Gordon', 'age': 66},
 {'name': 'Abraham Joshua Heschel', 'age': 65},
 {'name': 'Albert of Saxony', 'age': 74},
 {'name': 'Alessandro Achillini', 'age': 49},
 {'name': 'Alexander Gottlieb Baumgarten', 'age': 48},
 {'name': 'Alexius Meinong', 'age': 67},
 {'name': 'Alfred Firmin Loisy', 'age': 83},
 {'name': 'Alfred Korzybski', 'age': 71},
 {'name': 'Alfred North Whitehead', 'age': 86},
 {'name': 'Allan Bloom', 'age': 62},
 {'name': 'Anicius Manlius Severinus Boethius', 'age': 49},
 {'name': 'Antonio Labriola', 'age': 61},
 {'name': 'Antonio Rosmini-Serbati', 'age': 58},
 {'name': 'Antony Flew', 'age': 87},
 {'name': 'Armando Carlini', 'age': 81},
 {'name': 'Arthur O. Lovejoy', 'age': 89},
 {'name': 'Arthur Schopenhauer', 'age': 72},
 {'name': 'Arthur Stanley Eddington', 'age': 62},
 {'name': 'Ashvaghosha', 'age': 70},
 {'name': 'Auguste Comte', 'age': 59},
 {'name': 'Averroes', 'age': 72},
 {'name': 'Avicenna', 'age': 57},
 {'name': 'Bartolome de Medina', 'age': 5

Why do we care? The fact that we wrote in a  functional style, we can leverage parralelism

In [29]:
%%time
import multiprocess

def transform(x):
    print(f"Processing philosopher: {x.name}")
    time.sleep(.01)
    return {"name": x.name, "age": x.died - x.born}

pool = multiprocess.Pool()

result = pool.map(transform, philosophers)

result

Processing philosopher: Aaron David Gordon
Processing philosopher: Alfred North WhiteheadProcessing philosopher: Benedict de SpinozaProcessing philosopher: Arthur Schopenhauer
Processing philosopher: Carl Gustav HempelProcessing philosopher: Emmanuel LevinasProcessing philosopher: Frederick Robert TennantProcessing philosopher: Etienne Bonnot de Condillac
Processing philosopher: David Ross


Processing philosopher: Gaston Frommel



Processing philosopher: Abraham Joshua HeschelProcessing philosopher: Allan BloomProcessing philosopher: Charles Sanders Peirce
Processing philosopher: Erich FrankProcessing philosopher: Arthur Stanley EddingtonProcessing philosopher: Friedrich NietzscheProcessing philosopher: Bernard BosanquetProcessing philosopher: Denis DiderotProcessing philosopher: Etienne Gilson
Processing philosopher: Ge Hong







Processing philosopher: Albert of Saxony
Processing philosopher: Friedrich SchleiermacherProcessing philosopher: AshvaghoshaProcessing philosopher: Derek

Processing philosopher: Max SchelerProcessing philosopher: Ramon LlullProcessing philosopher: Muro KyusoProcessing philosopher: Pierre LerouxProcessing philosopher: Shao YongProcessing philosopher: Rudolf BultmannProcessing philosopher: Otto Friedrich von GierkeProcessing philosopher: Kumazawa Banzan







Processing philosopher: Marin MersenneProcessing philosopher: Louis-Alexandre Couturat

Processing philosopher: Max StirnerProcessing philosopher: Nathanael CulverwelProcessing philosopher: Reinhold NiebuhrProcessing philosopher: Shri AurobindoProcessing philosopher: Pierre NicoleProcessing philosopher: Rudolf CarnapProcessing philosopher: Otto NeurathProcessing philosopher: Leopold Kronecker







Processing philosopher: Marjorie GreneProcessing philosopher: Lu Jiuyuan

Processing philosopher: Michel FoucaultProcessing philosopher: Pierre Teilhard de ChardinProcessing philosopher: Rene DescartesProcessing philosopher: Sidney HookProcessing philosopher: Nicholas of CusaProcessing p

[{'name': 'Aaron David Gordon', 'age': 66},
 {'name': 'Abraham Joshua Heschel', 'age': 65},
 {'name': 'Albert of Saxony', 'age': 74},
 {'name': 'Alessandro Achillini', 'age': 49},
 {'name': 'Alexander Gottlieb Baumgarten', 'age': 48},
 {'name': 'Alexius Meinong', 'age': 67},
 {'name': 'Alfred Firmin Loisy', 'age': 83},
 {'name': 'Alfred Korzybski', 'age': 71},
 {'name': 'Alfred North Whitehead', 'age': 86},
 {'name': 'Allan Bloom', 'age': 62},
 {'name': 'Anicius Manlius Severinus Boethius', 'age': 49},
 {'name': 'Antonio Labriola', 'age': 61},
 {'name': 'Antonio Rosmini-Serbati', 'age': 58},
 {'name': 'Antony Flew', 'age': 87},
 {'name': 'Armando Carlini', 'age': 81},
 {'name': 'Arthur O. Lovejoy', 'age': 89},
 {'name': 'Arthur Schopenhauer', 'age': 72},
 {'name': 'Arthur Stanley Eddington', 'age': 62},
 {'name': 'Ashvaghosha', 'age': 70},
 {'name': 'Auguste Comte', 'age': 59},
 {'name': 'Averroes', 'age': 72},
 {'name': 'Avicenna', 'age': 57},
 {'name': 'Bartolome de Medina', 'age': 5

In [30]:
%%time
import multiprocess
import os

def transform(x):
    print(f"Process ID {os.getpid()} is processing philosopher: {x.name}")
    time.sleep(.01)
    return {"name": x.name, "age": x.died - x.born}

pool = multiprocess.Pool()

result = pool.map(transform, philosophers)

result

Process ID 93443 is processing philosopher: Aaron David GordonProcess ID 93444 is processing philosopher: Alfred North WhiteheadProcess ID 93445 is processing philosopher: Arthur SchopenhauerProcess ID 93446 is processing philosopher: Benedict de SpinozaProcess ID 93447 is processing philosopher: Carl Gustav HempelProcess ID 93448 is processing philosopher: David RossProcess ID 93449 is processing philosopher: Emmanuel LevinasProcess ID 93450 is processing philosopher: Etienne Bonnot de CondillacProcess ID 93451 is processing philosopher: Frederick Robert Tennant



Process ID 93452 is processing philosopher: Gaston Frommel





Process ID 93443 is processing philosopher: Abraham Joshua HeschelProcess ID 93444 is processing philosopher: Allan BloomProcess ID 93445 is processing philosopher: Arthur Stanley EddingtonProcess ID 93449 is processing philosopher: Erich FrankProcess ID 93450 is processing philosopher: Etienne GilsonProcess ID 93448 is processing philosopher: Denis DiderotProc

Process ID 93443 is processing philosopher: Ibn GabirolProcess ID 93447 is processing philosopher: John DeweyProcess ID 93446 is processing philosopher: George Trumbull LaddProcess ID 93445 is processing philosopher: Joseph PriestleyProcess ID 93448 is processing philosopher: John VennProcess ID 93444 is processing philosopher: Jean BuridanProcess ID 93452 is processing philosopher: Jacques Maritain
Process ID 93449 is processing philosopher: Henry Longueville ManselProcess ID 93451 is processing philosopher: Harry S. BroudyProcess ID 93450 is processing philosopher: Gottlob Frege








Process ID 93443 is processing philosopher: Immanuel KantProcess ID 93445 is processing philosopher: Josiah RoyceProcess ID 93446 is processing philosopher: George TyrrellProcess ID 93448 is processing philosopher: John WycliffeProcess ID 93447 is processing philosopher: John FiskeProcess ID 93451 is processing philosopher: Hatano SeiichiProcess ID 93452 is processing philosopher: Jaime Luciano Balmes

Process ID 93445 is processing philosopher: Yan YuanProcess ID 93450 is processing philosopher: Thomas ReidProcess ID 93448 is processing philosopher: Susanne K. LangerProcess ID 93447 is processing philosopher: William Kingdon CliffordProcess ID 93449 is processing philosopher: Wang Yangming




Process ID 93449 is processing philosopher: Watsuji TetsuroProcess ID 93448 is processing philosopher: TaixuProcess ID 93445 is processing philosopher: Zhang ZaiProcess ID 93447 is processing philosopher: William PaleyProcess ID 93450 is processing philosopher: Tommaso Campanella




Process ID 93445 is processing philosopher: Zhou DunyiProcess ID 93449 is processing philosopher: Werner HeisenbergProcess ID 93447 is processing philosopher: William Ralph IngeProcess ID 93450 is processing philosopher: Uriel AcostaProcess ID 93448 is processing philosopher: Tao Sheng




Process ID 93447 is processing philosopher: William WhewellProcess ID 93449 is processing philosopher: Wilfrid SellarsProcess 

[{'name': 'Aaron David Gordon', 'age': 66},
 {'name': 'Abraham Joshua Heschel', 'age': 65},
 {'name': 'Albert of Saxony', 'age': 74},
 {'name': 'Alessandro Achillini', 'age': 49},
 {'name': 'Alexander Gottlieb Baumgarten', 'age': 48},
 {'name': 'Alexius Meinong', 'age': 67},
 {'name': 'Alfred Firmin Loisy', 'age': 83},
 {'name': 'Alfred Korzybski', 'age': 71},
 {'name': 'Alfred North Whitehead', 'age': 86},
 {'name': 'Allan Bloom', 'age': 62},
 {'name': 'Anicius Manlius Severinus Boethius', 'age': 49},
 {'name': 'Antonio Labriola', 'age': 61},
 {'name': 'Antonio Rosmini-Serbati', 'age': 58},
 {'name': 'Antony Flew', 'age': 87},
 {'name': 'Armando Carlini', 'age': 81},
 {'name': 'Arthur O. Lovejoy', 'age': 89},
 {'name': 'Arthur Schopenhauer', 'age': 72},
 {'name': 'Arthur Stanley Eddington', 'age': 62},
 {'name': 'Ashvaghosha', 'age': 70},
 {'name': 'Auguste Comte', 'age': 59},
 {'name': 'Averroes', 'age': 72},
 {'name': 'Avicenna', 'age': 57},
 {'name': 'Bartolome de Medina', 'age': 5

In [31]:
%%time
import multiprocess
import os

def transform(x):
    print(f"Process ID {os.getpid()} is processing philosopher: {x.name}")
    time.sleep(.01)
    return {"name": x.name, "age": x.died - x.born}

pool = multiprocess.Pool(processes=5)

result = pool.map(transform, philosophers)

result

Process ID 93453 is processing philosopher: Aaron David GordonProcess ID 93454 is processing philosopher: Arthur O. LovejoyProcess ID 93455 is processing philosopher: Blaise PascalProcess ID 93456 is processing philosopher: Eduard von HartmannProcess ID 93457 is processing philosopher: Francisco Suarez




Process ID 93454 is processing philosopher: Arthur SchopenhauerProcess ID 93453 is processing philosopher: Abraham Joshua HeschelProcess ID 93457 is processing philosopher: Franciscus HemsterhuisProcess ID 93456 is processing philosopher: Edward CairdProcess ID 93455 is processing philosopher: Blessed John Duns Scotus




Process ID 93457 is processing philosopher: Franz BrentanoProcess ID 93454 is processing philosopher: Arthur Stanley EddingtonProcess ID 93453 is processing philosopher: Albert of SaxonyProcess ID 93455 is processing philosopher: Carl Gustav Hempel



Process ID 93456 is processing philosopher: Edward Westermarck
Process ID 93454 is processing philosopher: Ashvaghos

Process ID 93456 is processing philosopher: John Neville KeynesProcess ID 93453 is processing philosopher: Harry S. BroudyProcess ID 93455 is processing philosopher: George TyrrellProcess ID 93454 is processing philosopher: Ibn FalaqueraProcess ID 93457 is processing philosopher: James Martineau




Process ID 93456 is processing philosopher: John Scott HaldaneProcess ID 93453 is processing philosopher: Hatano SeiichiProcess ID 93454 is processing philosopher: Ibn GabirolProcess ID 93455 is processing philosopher: Geraud de CordemoyProcess ID 93457 is processing philosopher: Jean Baudrillard




Process ID 93453 is processing philosopher: Hayashi RazanProcess ID 93455 is processing philosopher: Giambattista VicoProcess ID 93456 is processing philosopher: John Scotus ErigenaProcess ID 93454 is processing philosopher: Immanuel KantProcess ID 93457 is processing philosopher: Jean Buridan




Process ID 93453 is processing philosopher: He YanProcess ID 93454 is processing philosopher: Inou

Process ID 93455 is processing philosopher: Thomas Henry HuxleyProcess ID 93453 is processing philosopher: Willard Van Orman QuineProcess ID 93457 is processing philosopher: al-FarabiProcess ID 93454 is processing philosopher: Shao Yong



Process ID 93457 is processing philosopher: al-GhazaliProcess ID 93453 is processing philosopher: William GodwinProcess ID 93454 is processing philosopher: Shri AurobindoProcess ID 93455 is processing philosopher: Thomas Hobbes



Process ID 93455 is processing philosopher: Thomas ReidProcess ID 93453 is processing philosopher: William JamesProcess ID 93454 is processing philosopher: Sidney Hook


Process ID 93455 is processing philosopher: Tommaso CampanellaProcess ID 93454 is processing philosopher: Simon FoucherProcess ID 93453 is processing philosopher: William Kingdon Clifford


Process ID 93454 is processing philosopher: Simone WeilProcess ID 93455 is processing philosopher: Uriel AcostaProcess ID 93453 is processing philosopher: William Paley


[{'name': 'Aaron David Gordon', 'age': 66},
 {'name': 'Abraham Joshua Heschel', 'age': 65},
 {'name': 'Albert of Saxony', 'age': 74},
 {'name': 'Alessandro Achillini', 'age': 49},
 {'name': 'Alexander Gottlieb Baumgarten', 'age': 48},
 {'name': 'Alexius Meinong', 'age': 67},
 {'name': 'Alfred Firmin Loisy', 'age': 83},
 {'name': 'Alfred Korzybski', 'age': 71},
 {'name': 'Alfred North Whitehead', 'age': 86},
 {'name': 'Allan Bloom', 'age': 62},
 {'name': 'Anicius Manlius Severinus Boethius', 'age': 49},
 {'name': 'Antonio Labriola', 'age': 61},
 {'name': 'Antonio Rosmini-Serbati', 'age': 58},
 {'name': 'Antony Flew', 'age': 87},
 {'name': 'Armando Carlini', 'age': 81},
 {'name': 'Arthur O. Lovejoy', 'age': 89},
 {'name': 'Arthur Schopenhauer', 'age': 72},
 {'name': 'Arthur Stanley Eddington', 'age': 62},
 {'name': 'Ashvaghosha', 'age': 70},
 {'name': 'Auguste Comte', 'age': 59},
 {'name': 'Averroes', 'age': 72},
 {'name': 'Avicenna', 'age': 57},
 {'name': 'Bartolome de Medina', 'age': 5

### Multiprocess vs Multiprocessing

Just a note here that I am using the package `multiprocess` above, but in practice, you will almost always want to use the package that is shipped with Python `multiprocessing`. The reason I am not using `mutliprocessing` above, but instead the package `multiprocess`, which has to be separately pip installed is that `mutliprocessing` relies heavily on the the `__main__` attribute which is not precent when running in a shell or on Jupyter Notebook.

In [32]:
%%time
# Example of multiprocessing
!python multiprocessing_example.py

Process ID 93463 is processing philosopher: Aaron David Gordon
Process ID 93463 is processing philosopher: Abraham Joshua Heschel
Process ID 93463 is processing philosopher: Albert of Saxony
Process ID 93462 is processing philosopher: Arthur O. Lovejoy
Process ID 93464 is processing philosopher: Blaise Pascal
Process ID 93460 is processing philosopher: Eduard von Hartmann
Process ID 93463 is processing philosopher: Alessandro Achillini
Process ID 93462 is processing philosopher: Arthur Schopenhauer
Process ID 93461 is processing philosopher: Francisco Suarez
Process ID 93464 is processing philosopher: Blessed John Duns Scotus
Process ID 93460 is processing philosopher: Edward Caird
Process ID 93463 is processing philosopher: Alexander Gottlieb Baumgarten
Process ID 93462 is processing philosopher: Arthur Stanley Eddington
Process ID 93461 is processing philosopher: Franciscus Hemsterhuis
Process ID 93464 is processing philosopher: Carl Gustav Hempel
Process ID 93460 is processing philo

Process ID 93463 is processing philosopher: John of Paris
Process ID 93461 is processing philosopher: John Stuart Mill
Process ID 93462 is processing philosopher: He Yan
Process ID 93464 is processing philosopher: Jean-Jacques Rousseau
Process ID 93460 is processing philosopher: Inoue Tetsujiro
Process ID 93463 is processing philosopher: John of Salisbury
Process ID 93461 is processing philosopher: John Venn
Process ID 93462 is processing philosopher: Leopold Kronecker
Process ID 93464 is processing philosopher: Marin Mersenne
Process ID 93463 is processing philosopher: Jonathan Edwards
Process ID 93460 is processing philosopher: Muro Kyuso
Process ID 93462 is processing philosopher: Levi ben Gershom
Process ID 93461 is processing philosopher: Pierre Laromiguiere
Process ID 93464 is processing philosopher: Marjorie Grene
Process ID 93463 is processing philosopher: Jose Ortega y Gasset
Process ID 93460 is processing philosopher: Nathanael Culverwel
Process ID 93462 is processing philoso

CPU times: user 39.5 ms, sys: 21.2 ms, total: 60.6 ms
Wall time: 1.78 s


## Concurrent.futures


More modern way of handling multiprocessing in Python

In [33]:
%%time
!python concurrent_example.py

Process ID 93472 is processing philosopher: Aaron David Gordon
Process ID 93468 is processing philosopher: Abraham Joshua Heschel
Process ID 93472 is processing philosopher: Albert of Saxony
Process ID 93467 is processing philosopher: Alessandro Achillini
Process ID 93468 is processing philosopher: Alexander Gottlieb Baumgarten
Process ID 93469 is processing philosopher: Alexius Meinong
Process ID 93472 is processing philosopher: Alfred Firmin Loisy
Process ID 93475 is processing philosopher: Alfred Korzybski
Process ID 93467 is processing philosopher: Alfred North Whitehead
Process ID 93468 is processing philosopher: Allan Bloom
Process ID 93470 is processing philosopher: Anicius Manlius Severinus Boethius
Process ID 93469 is processing philosopher: Antonio Labriola
Process ID 93473 is processing philosopher: Antonio Rosmini-Serbati
Process ID 93472 is processing philosopher: Antony Flew
Process ID 93475 is processing philosopher: Armando Carlini
Process ID 93467 is processing philoso

Process ID 93475 is processing philosopher: Josiah Royce
Process ID 93472 is processing philosopher: Julian Huxley
Process ID 93473 is processing philosopher: Justus Lipsius
Process ID 93471 is processing philosopher: Karl Jaspers
Process ID 93469 is processing philosopher: Karl Marx
Process ID 93476 is processing philosopher: Karl Popper
Process ID 93468 is processing philosopher: Kumazawa Banzan
Process ID 93474 is processing philosopher: Leopold Kronecker
Process ID 93467 is processing philosopher: Levi ben Gershom
Process ID 93470 is processing philosopher: Li Ao
Process ID 93475 is processing philosopher: Liang Shuming
Process ID 93472 is processing philosopher: Lorenzo Valla
Process ID 93473 is processing philosopher: Louis Althusser
Process ID 93471 is processing philosopher: Louis Lavelle
Process ID 93470 is processing philosopher: Louis-Alexandre Couturat
Process ID 93474 is processing philosopher: Lu Jiuyuan
Process ID 93468 is processing philosopher: Lucien Levy-Bruhl
Proces

CPU times: user 25.9 ms, sys: 16.1 ms, total: 42 ms
Wall time: 1.97 s


Why concurrent.futures over multiprocessing?

One answer, it allows for multithreading...

In [34]:
%%time
!python concurrent_example.py

Process ID 93481 is processing philosopher: Aaron David Gordon
Process ID 93481 is processing philosopher: Abraham Joshua Heschel
Process ID 93481 is processing philosopher: Albert of Saxony
Process ID 93481 is processing philosopher: Alessandro Achillini
Process ID 93481 is processing philosopher: Alexander Gottlieb Baumgarten
Process ID 93481 is processing philosopher: Alexius Meinong
Process ID 93481 is processing philosopher: Alfred Firmin Loisy
Process ID 93479 is processing philosopher: Alfred Korzybski
Process ID 93483 is processing philosopher: Alfred North Whitehead
Process ID 93479 is processing philosopher: Allan Bloom
Process ID 93481 is processing philosopher: Anicius Manlius Severinus Boethius
Process ID 93484 is processing philosopher: Antonio Labriola
Process ID 93483 is processing philosopher: Antonio Rosmini-Serbati
Process ID 93485 is processing philosopher: Antony Flew
Process ID 93479 is processing philosopher: Armando Carlini
Process ID 93481 is processing philoso

Process ID 93481 is processing philosopher: John Wycliffe
Process ID 93488 is processing philosopher: John of Paris
Process ID 93479 is processing philosopher: John of Salisbury
Process ID 93485 is processing philosopher: Jonathan Edwards
Process ID 93482 is processing philosopher: Jose Ortega y Gasset
Process ID 93487 is processing philosopher: Jose Vasconcelos
Process ID 93486 is processing philosopher: Joseph Butler
Process ID 93483 is processing philosopher: Joseph Priestley
Process ID 93481 is processing philosopher: Josiah Royce
Process ID 93480 is processing philosopher: Julian Huxley
Process ID 93484 is processing philosopher: Justus Lipsius
Process ID 93488 is processing philosopher: Karl Jaspers
Process ID 93479 is processing philosopher: Karl Marx
Process ID 93482 is processing philosopher: Karl Popper
Process ID 93487 is processing philosopher: Kumazawa Banzan
Process ID 93486 is processing philosopher: Leopold Kronecker
Process ID 93485 is processing philosopher: Levi ben 

CPU times: user 27.8 ms, sys: 16.5 ms, total: 44.3 ms
Wall time: 1.82 s


### Multiprocessing vs Concurrent.Futures and the GIL

Python has something called the [Global Interpreter Lock (GIL)](https://realpython.com/python-gil/). The GIL prevents Python code from being executed in more than one thread at a time. So you may be asking yourself, how were we able to run `concurrent_example.py` above with `ThreadPoolExecutor` and get a faster time than with `ProcessPoolExecutor`. That is because our `time.sleep` is an I/O command that is blocking the thread, thereby allowing other threads to process.

So, to regards to the question, "when should I use multiprocessing and when should I use concurrent.futures?" the answer is that you can use either, but you are probably better off using `concurrent.futures` every time. This is because `concurrent.futures` gives you access to both the `ThreadPoolExecutor` and the `ProcessPoolExecutor`, which each have their own time and place for use. Additionally, `ProcessPoolExecutor` is just a wrapper around `multiprocessing` (see [docs](https://docs.python.org/3/library/concurrent.futures.html#processpoolexecutor)), and there is no equivalent to `ThreadPoolExecutor` in `multiprocessing`.


### The End of the GIL?

This title is a bit click baity. No one is proposing we do away with the GIL entirely. However, there is a PEP currently under consideration ([PEP-684](https://peps.python.org/pep-0684/)) that proposes isolating the GIL to the interpreter level, not the process level. This would allow multiple interpreters to run in a process independently and help unlock true multi-core parallelism in Python. 

## Resources

* [Overview of Functional Programming Generally](https://www.guru99.com/functional-programming-tutorial.html)
* [Functional Programming in Python](https://realpython.com/python-functional-programming/)
* [Philosopher Data Source](https://www.kaggle.com/datasets/matheusdalbuquerque/philosophers-dataset?resource=download)
* [Python GIL](https://realpython.com/python-gil/)
